In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
df = pd.read_csv("../data/final_version.csv")

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df

In [ ]:
df_filtré = df.drop(columns=['lib_commune', 'code_commune_INSEE', "latitude","longitude"])

In [ ]:
data_encoded = pd.get_dummies(df_filtré)

In [ ]:
X = data_encoded.drop(columns=['% Voix/Exp'])
y = data_encoded['% Voix/Exp']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
RFR

In [11]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

<br><br><br><br>
Essai grid_search

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10]
}

In [ ]:
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
best_params = grid_search.best_params_
print("Meilleurs hyperparamètres:", best_params)

In [ ]:
rf_model_tuned = RandomForestRegressor(random_state=42, **best_params)
rf_model_tuned.fit(X_train, y_train)

In [ ]:
y_pred_tuned = rf_model_tuned.predict(X_test)

In [ ]:
mse_tuned = mean_squared_error(y_test, y_pred_tuned)
r2_tuned = r2_score(y_test, y_pred_tuned)

print("Mean Squared Error (modèle optimisé):", mse_tuned)
print("R-squared (modèle optimisé):", r2_tuned)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y_test, y_pred_tuned)
plt.show()